# Redes convolucionales
Estos ejemplo implementa y entrena modelos de redes neuronales convolucionales y densas utilizando TensorFlow y Keras. La idea es comparar el comportamiento de un tipo de Red Neuronal frente al otro, para verificar la performance de cada uno en el mismo escenario.

## Importar librerias

In [ ]:
import numpy as np
import time
import matplotlib.pyplot as plt

import tensorflow as tf

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D

from tensorflow.keras.metrics import MSE
from tensorflow.keras.optimizers import Adadelta
from tensorflow.keras.losses import categorical_crossentropy

from tensorflow.keras.utils import to_categorical

# El dataset CIFAR-10, contiene imágenes de 10 categorías diferentes.
from tensorflow.keras.datasets import cifar10

In [ ]:
lr = 1.0
epochs = 5
batch_size = 32
np.random.seed(14)

## Cargar y visualizar el dataset

In [ ]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
n_classes = np.max(np.unique(y_train)) + 1

print("Verificamos la cantidad de clases = %d" % (n_classes))

In [ ]:
num = 25
images = x_train[:num]
labels = y_train[:num]
num_row = 5
num_col = 5
fig, axes = plt.subplots(num_row, num_col, figsize=(1.5*num_col,2*num_row))
for i in range(num):
    ax = axes[i//num_col, i%num_col]
    ax.imshow(images[i], cmap='gray')
    ax.set_title('Label: {}'.format(labels[i]))
    ax.axis('off')
plt.tight_layout()
plt.show()

## Preparacion de los datos

In [ ]:
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

In [ ]:
y_train = to_categorical(y_train, n_classes)
y_test = to_categorical(y_test, n_classes) #one-hot encoding

In [ ]:
# Se realiza un reshape de todas las imágenes en el conjunto de datos de
# entrenamiento para transformarlas de matrices tridimensionales a vectores
# unidimensionales:
x_train = x_train.reshape(x_train.shape[0], np.prod(x_train.shape[1:]))
x_test = x_test.reshape(x_test.shape[0], np.prod(x_test.shape[1:]))

In [ ]:
# Dimensiones del conjunto de datos de entrenamiento:
# El primer número representa la cantidad  de ejemplos o imágenes en
# el conjunto de datos de entrenamiento.
# El segundo el número de características por ejemplo.
x_train.shape

## Modelo de redes densas

In [ ]:
# Model
#---------------------------------------------------------------------#
input_layer = Input(shape=x_train.shape[1:])
dense_1 = Dense(500, activation='relu') (input_layer)
dropout_1 = Dropout(0.25) (dense_1)
dense_2 = Dense(500, activation='relu') (dropout_1)
dropout_2 = Dropout(0.25) (dense_2)
dense_3 = Dense(500, activation='relu') (dropout_2)
dropout_3 = Dropout(0.25) (dense_3)
output_layer = Dense(n_classes, activation='softmax') (dropout_3)
#---------------------------------------------------------------------#
model_dense = Model(input_layer, output_layer)

In [ ]:
Adadelta_optimizer = Adadelta(learning_rate=lr, rho=0.95)
model_dense.compile(optimizer=Adadelta_optimizer, loss='categorical_crossentropy', metrics=['acc', 'mse'])
model_dense.summary()

## Entrenar el modelo

In [ ]:
start_time = time.time()
history_dense = model_dense.fit(x_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(x_test, y_test), shuffle=True, verbose=1)
end_time = time.time()
print('\nElapsed Dense Model training time: {:.5f} seconds'.format(end_time-start_time))

## Modelo de redes convolucionales

In [ ]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
n_classes = np.max(np.unique(y_train)) + 1

In [ ]:
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

In [ ]:
y_train = to_categorical(y_train, n_classes)
y_test = to_categorical(y_test, n_classes) #one-hot encoding

In [ ]:
x_train.shape

In [ ]:
# Model
#---------------------------------------------------------------------#
input_layer = Input(shape=x_train.shape[1:])
conv_1 = Conv2D(32, (3, 3), activation='relu') (input_layer)
conv_2 = Conv2D(64, (3, 3), activation='relu') (conv_1)
pool_1 = MaxPooling2D(pool_size=(2, 2)) (conv_2)
dropout_1 = Dropout(0.25) (pool_1)
flatten_1 = Flatten() (dropout_1)
dense_1 = Dense(100, activation='relu') (flatten_1)
dropout_2 = Dropout(0.25) (dense_1)
output_layer = Dense(n_classes, activation='softmax') (dropout_2)
#---------------------------------------------------------------------#
model_conv = Model(input_layer, output_layer)

In [ ]:
model_conv.compile(optimizer=Adadelta_optimizer, loss='categorical_crossentropy', metrics=['acc', 'mse'])
model_conv.summary()

In [ ]:
start_time = time.time()
history_conv = model_conv.fit(x_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(x_test, y_test), shuffle=True, verbose=1)
end_time = time.time()
print('\nElapsed Dense Model training time: {:.5f} seconds'.format(end_time-start_time))

## Evaluar el proceso de entrenamiento

In [ ]:
f = plt.figure(figsize=(10,8))

plt.subplot(1,2,1)
plt.plot(history_dense.history['acc'], linewidth=3, label='Train Accuracy')
plt.plot(history_dense.history['val_acc'], linewidth=3, label='Val Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.axis([0, 4, 0, 1])
plt.legend(loc='lower right')

plt.subplot(1,2,2)
plt.plot(history_conv.history['acc'], linewidth=3, label='Train Accuracy')
plt.plot(history_conv.history['val_acc'], linewidth=3, label='Val Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.axis([0, 4, 0, 1])
plt.legend(loc='lower right')

plt.show()

In [ ]:
f = plt.figure(figsize=(10,8))

plt.subplot(1,2,1)
plt.plot(history_dense.history['mse'], linewidth=3, label='Train Accuracy')
plt.plot(history_dense.history['val_mse'], linewidth=3, label='Val Accuracy')
plt.xlabel('Epochs')
plt.ylabel('MSE')
plt.axis([0, 4, 0, 0.1])
plt.legend(loc='lower right')

plt.subplot(1,2,2)
plt.plot(history_conv.history['mse'], linewidth=3, label='Train Accuracy')
plt.plot(history_conv.history['val_mse'], linewidth=3, label='Val Accuracy')
plt.xlabel('Epochs')
plt.ylabel('MSE')
plt.axis([0, 4, 0, 0.1])
plt.legend(loc='lower right')

plt.show()

Las redes convolucionales muestran generalmente un mejor desempeño en la clasificación de imágenes debido a su capacidad para extraer características relevantes y reducir la dimensionalidad de manera eficiente.